In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels

from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder

from GEMS_TCO import configuration as config
from GEMS_TCO import data_preprocess as dmbh

import os
from sklearn.neighbors import BallTree

# Load data

In [2]:
lon_s = 123
lon_e = 133
step_lat = 0.044
step_lon = 0.063

lat_coords = np.arange( 5 -0.044- 0.0002, 0 -0.044, -0.044)
lon_coords = np.arange( lon_e-step_lon- 0.0002, lon_s-step_lon, -step_lon)

# Apply the shift as in the original code
# These are the unique lat/lon values for the "center_points" grid
final_lat_values = lat_coords + step_lat 
final_lon_values = lon_coords + step_lon 

# Create 2D grid with broadcasting
#decrement = 0.00012
decrement = 0 
lat_grid = final_lat_values[:, None] + np.arange(len(final_lon_values)) * decrement  # shape: (228, 152)


mac_data_path = config.mac_data_load_path
years = [2024]  # years = [2023,2024]
months = list( range(7,8))
year = years[0]
month = months[0]
month_str = f"{month:02d}"  
filename = f"pickle_2024/orbit_map{str(year)[2:]}_{month_str}.pkl"
picklefile_path = Path(mac_data_path) / filename
print(picklefile_path)

with open(picklefile_path, 'rb') as pickle_file:
    data_map_hour = pickle.load(pickle_file)

# Base file path and settings
# base_path = "C:\\Users\\joonw\\TCO\\GEMS_data"    MSI notebook

mac_data_path = config.mac_data_load_path
lat_start, lat_end, lon_start, lon_end = 0, 5, 123, 133
step_lat, step_lon = 0.044, 0.063

# df = pd.read_csv("C:\\Users\\joonw\\TCO\\GEMS_data\\data_2024\\data_24_07_0131_N510_E110120.csv")  MSI notebook
df = pd.read_csv("/Users/joonwonlee/Documents/GEMS_DATA/data_2024/data_24_07_0131_N05_E123133.csv")  # MAC


/Users/joonwonlee/Documents/GEMS_DATA/pickle_2024/orbit_map24_07.pkl


In [ ]:
''' 
instance = dmbh.center_matching_hour(df, lat_start, lat_end, lon_start, lon_end)

for year in years:        # years = [2023,2024]
    for month in months:  
        try:
            # load pickle (dense ORI data)
            pickle_path = os.path.join(mac_data_path, f'pickle_{year}')
            input_filename = f"orbit_map{str(year)[2:]}_{month_str}.pkl"
            input_filepath = os.path.join(pickle_path, input_filename)
            with open(input_filepath, 'rb') as pickle_file:
                loaded_map = pickle.load(pickle_file)
            center_points = instance.make_center_points_wo_calibration(step_lat = step_lat, step_lon= step_lon)
            coarse_cen_map = instance.coarse_by_center(loaded_map, center_points)

            # Save pickle (coarse data)
            output_filename = f"coarse_cen_map_without_decrement_latitude{str(year)[2:]}_{month_str}.pkl"
            output_filepath = os.path.join(pickle_path, output_filename)
            with open(output_filepath, 'wb') as pickle_file:
                pickle.dump(coarse_cen_map, pickle_file)
            
            print(f"Successfully processed and saved data for year {str(year)[2:]} month {month_str}.")
        except FileNotFoundError:
            print(f"Warning: File {input_filename} not found. Skipping.")
        except Exception as e:
            print(f"Error processing file {input_filename}: {e}")
'''

In [7]:
pickle_path = os.path.join(mac_data_path, f'pickle_{year}')
output_filename = f"coarse_cen_map_without_decrement_latitude{str(year)[2:]}_{month_str}.pkl"
output_filepath = os.path.join(pickle_path, output_filename)

with open(output_filepath, 'rb') as pickle_file:
    cbmap_ori = pickle.load(pickle_file)


load_data_instance = GEMS_TCO.load_data('')

df_day_list = []
df_day_map_list = []
for i in range(31):
    cur_map, cur_df =load_data_instance.load_working_data_byday_wo_mm(cbmap_ori,[i*8, (i+1)*8])
    df_day_list.append( cur_df )
    df_day_map_list.append( cur_map )

In [12]:
df_day_map_list[0]['y24m07day01_hm00:52'].dtype

torch.float32

# Apply laplacian filter

In [ ]:
import torch
import numpy as np
import torch.nn.functional as F

def subset_tensor(df_tensor: torch.Tensor) -> torch.Tensor:
    """
    Subsets a tensor to a specific lat/lon range.
    Columns are assumed to be [lat, lon, ozone, time].
    """
    lat_mask = (df_tensor[:, 0] >= 0) & (df_tensor[:, 0] <= 2.9)
    lon_mask = (df_tensor[:, 1] >= 123.5) & (df_tensor[:, 1] <= 131.7)
    
    # We apply the mask to the full tensor
    df_sub = df_tensor[lat_mask & lon_mask].clone()
    return df_sub

def apply_laplacian_2d_valid_tensor(df_tensor: torch.Tensor) -> torch.Tensor:
    """
    Applies a 2D discrete Laplacian filter using 'mode=valid' on a PyTorch tensor.
    The input tensor columns are assumed to be [lat, lon, ozone, time].
    """
    # Extract unique coordinates and data
    unique_lats = torch.unique(df_tensor[:, 0])
    unique_lons = torch.unique(df_tensor[:, 1])
    
    lat_count = unique_lats.size(0)
    lon_count = unique_lons.size(0)
    
    if df_tensor.size(0) != lat_count * lon_count:
        raise ValueError("Tensor size does not match the grid dimensions (lat * lon).")

    ozone_data = df_tensor[:, 2].reshape(lat_count, lon_count)

    # Define the 3x3 discrete Laplacian kernel as a PyTorch tensor
    laplacian_kernel = torch.tensor([[0, 1, 0],
                                     [1, -4, 1],
                                     [0, 1, 0]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    # Reshape ozone data for conv2d: (batch_size, channels, height, width)
    ozone_data_reshaped = ozone_data.unsqueeze(0).unsqueeze(0)

    # Apply convolution
    laplacian_grid_cropped = F.conv2d(ozone_data_reshaped, laplacian_kernel, padding='valid')
    laplacian_values = laplacian_grid_cropped.squeeze().flatten()

    # Determine the new, cropped coordinates
    cropped_lats = unique_lats[1:-1]
    cropped_lons = unique_lons[1:-1]
    
    if len(cropped_lats) == 0 or len(cropped_lons) == 0:
        return torch.empty(0, 4)

    # Create a new meshgrid for the cropped coordinates
    new_lon_grid, new_lat_grid = torch.meshgrid(cropped_lons, cropped_lats, indexing='xy')
    
    # Preserve time from the original tensor
    time_value = df_tensor[0, 3].repeat(laplacian_values.size(0))

    # Create the new tensor
    new_tensor = torch.stack([new_lat_grid.flatten(), new_lon_grid.flatten(), laplacian_values, time_value], dim=1)
    
    return new_tensor


tensor([[ 7.2000e-02,  1.2361e+02, -6.5655e+00,  2.1000e+01],
        [ 7.2000e-02,  1.2368e+02, -3.0364e+00,  2.1000e+01],
        [ 7.2000e-02,  1.2374e+02,  2.0616e+01,  2.1000e+01],
        ...,
        [ 2.8440e+00,  1.3149e+02,  5.0479e+00,  2.1000e+01],
        [ 2.8440e+00,  1.3155e+02, -8.7733e+00,  2.1000e+01],
        [ 2.8440e+00,  1.3161e+02, -3.4899e+00,  2.1000e+01]])

In [ ]:
processed_tensor_map = []
processed_df = []

for day_map in df_day_map_list:
    
    tensors_to_aggregate = []
    
    for key, tensor in day_map.items():
        # Step 1: Subset the tensor.
        subsetted = subset_tensor(tensor)
        
        if subsetted.size(0) > 0:
            laplacian_applied = apply_laplacian_2d_valid_tensor(subsetted)
            if laplacian_applied.size(0) > 0:
                tensors_to_aggregate.append(laplacian_applied)

    # Step 3: Concatenate all the processed tensors from the 8-hour period.
    if tensors_to_aggregate:
        aggregated_day_tensor = torch.cat(tensors_to_aggregate, dim=0)
        processed_df.append(aggregated_day_tensor)
        
    # The 'processed_tensors' list remains the same, storing the dictionary for each day.
    processed_tensor_map.append(day_map)

# Now, processed_df2 will contain a single aggregated tensor for each day (8-hour period).
# You can inspect the results:
print("Number of aggregated day tensors:", len(processed_df))
if processed_df:
    print("Shape of the first aggregated tensor:", processed_df[0].shape)
    print("First aggregated tensor head:")
    print(processed_df[0][:5])



Number of aggregated day tensors: 31
Shape of the first aggregated tensor: torch.Size([65536, 4])
First aggregated tensor head:
tensor([[ 7.2000e-02,  1.2361e+02, -6.5655e+00,  2.1000e+01],
        [ 7.2000e-02,  1.2368e+02, -3.0364e+00,  2.1000e+01],
        [ 7.2000e-02,  1.2374e+02,  2.0616e+01,  2.1000e+01],
        [ 7.2000e-02,  1.2380e+02, -4.3127e+00,  2.1000e+01],
        [ 7.2000e-02,  1.2386e+02, -1.0213e+00,  2.1000e+01]])


# Multivariate Debiased Whittle Likelihood

In [44]:
import numpy as np
import cmath
import pandas as pd
import time
import torch
import torch.nn.functional as F

def cgn(u, n1, n2):
    """
    Computes a 2D Bartlett window function (triangular window).
    
    Args:
        u (tuple): A tuple of lag indices (u1, u2) as torch.Tensors.
        n1 (int): The number of samples in the first dimension.
        n2 (int): The number of samples in the second dimension.
        
    Returns:
        torch.Tensor: The window value.
    """
    u1, u2 = u
    # Use torch operations for element-wise calculation
    return (1 - torch.abs(u1) / n1) * (1 - torch.abs(u2) / n2) 

def cov_x(u1, u2, t, params):
    """
    Computes the spatio-temporal autocovariance of the original process.
    
    Args:
        u1 (torch.Tensor): The first spatial lag.
        u2 (torch.Tensor): The second spatial lag.
        t (torch.Tensor): The temporal lag.
        params (list): A list of parameters for the covariance function.
        
    Returns:
        torch.Tensor: The autocovariance value.
    """
    sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params
    
    distance = (u1 / range_lat - advec_lat * t)**2 + (u2 / range_lon - advec_lon * t)**2 + (beta * t)**2

    # Using torch.where for conditional logic on tensors
    # This avoids issues with distance == 0
    return torch.where(distance != 0, sigmasq * torch.exp(-(distance)), sigmasq + nugget)

def cov_laplacian(u1, u2, t, params):
    """
    Computes the autocovariance of the Laplacian-filtered process.
    
    Args:
        u1 (torch.Tensor): The first lag index.
        u2 (torch.Tensor): The second lag index.
        t (torch.Tensor): The temporal lag.
        params (list): A list of parameters for the covariance function.
        
    Returns:
        torch.Tensor: The autocovariance value of the filtered process.
    """
    delta1, delta2 = 0.044, 0.063
    
    # Define the 5-point stencil of the discrete Laplacian
    stencil_weights = {(0, 0): -4, (0, 1): 1, (0, -1): 1, (1, 0): 1, (-1, 0): 1}
    
    # Initialize cov as a tensor of the correct shape to handle broadcasting
    cov = torch.zeros_like(u1, dtype=torch.float32) # Ensure dtype matches
    # Iterate through all pairs of points in the stencil
    for (a, b), w_ab in stencil_weights.items():
        for (c, d), w_cd in stencil_weights.items():
            # Calculate the effective lag vector
            lag_x = (u1 + a - c) * delta1
            lag_y = (u2 + b - d) * delta2

            # Add the weighted covariance term
            cov += w_ab * w_cd * cov_x(lag_x, lag_y, t, params)
    return cov

def cn_bar(u1, u2, t, params, n1, n2):
    """
    Computes the periodicized autocovariance by multiplying the
    Laplacian covariance with a 2D Bartlett window.
    
    Args:
        u1 (torch.Tensor): The first lag index.
        u2 (torch.Tensor): The second lag index.
        t (torch.Tensor): The temporal lag.
        params (list): Model parameters.
        n1 (int): The number of samples in the first dimension.
        n2 (int): The number of samples in the second dimension.
        
    Returns:
        torch.Tensor: The periodicized and windowed autocovariance value.
    """
    # u1 and u2 are now tensors
    u = (u1, u2)
    return cov_laplacian(u1, u2, t, params) * cgn(u, n1, n2)

def expected_periodogram_fft_multivariate(params, n1, n2, p):
    """
    Computes the multivariate expected periodogram for ALL frequencies using a 2D FFT.
    This method is much faster than the direct summation.
    
    Args:
        params (list): Model parameters.
        n1 (int): The number of samples in the first spatial dimension.
        n2 (int): The number of samples in the second dimension.
        p (int): The number of multivariate components.
        
    Returns:
        torch.Tensor: A 4D tensor (n1, n2, p, p) of expected periodogram matrices.
    """
    delta1, delta2 = 0.044, 0.063
    
    # Create a 4D tensor to hold the term c_g,n * c_X
    # Shape will be (n1, n2, p, p)
    product_tensor = torch.zeros((n1, n2, p, p), dtype=torch.complex64)
    
    # Define time lags based on the number of components
    t_lags = torch.arange(p, dtype=torch.float32)
    
    # Using torch.meshgrid to create tensors for u1 and u2 to enable vectorized operations
    u1_mesh, u2_mesh = torch.meshgrid(torch.arange(n1, dtype=torch.float32), torch.arange(n2, dtype=torch.float32), indexing='ij')
    
    for q in range(p):
        for r in range(p):
            # Temporal lag
            t = t_lags[q] - t_lags[r]
            
            # Compute the windowed autocovariance product c_g,n * c_X
            # The formula in the paper is a sum over u, so we're building the term for each u
            product_tensor[:, :, q, r] = cn_bar(u1_mesh, u2_mesh, t, params, n1, n2)
            
    # Perform the 2D FFT on the spatial dimensions for each component pair
    fft_result = torch.fft.fft2(product_tensor, dim=(-4, -3))
    # fft_result shape is (n1, n2, p, p)

    # Normalization factor from the paper
    normalization_factor = (delta1 * delta2) / (2 * cmath.pi)**2

    # delta 1 and delta 2 are dx1 dx2 in continuous integral. 
    
    expected_periodogram_tensor = fft_result * normalization_factor

    # Use torch.fft.fftshift
    return expected_periodogram_tensor


def generate_Jvector(tensor_list, lat_col=0, lon_col=1, val_col=2):
    """
    Generates a 3D tensor of DFT vectors from a list of tensors.
    
    Args:
        tensor_list (list): A list of tensors, where each tensor represents one
                            multivariate component.
        lat_col (int): Index of the latitude column.
        lon_col (int): Index of the longitude column.
        val_col (int): Index of the value column (e.g., 'laplacian').
        
    Returns:
        torch.Tensor: A 3D tensor (n1, n2, p) of DFT vectors for all frequencies.
    """
    p = len(tensor_list)
    if p == 0:
        return torch.empty(0, 0, 0)
        
    fft_results = []
    
    # Determine grid dimensions from the first tensor
    unique_lats = torch.unique(tensor_list[0][:, lat_col])
    unique_lons = torch.unique(tensor_list[0][:, lon_col])
    n1 = len(unique_lats)
    n2 = len(unique_lons)
    
    for tensor in tensor_list:
        # Create a grid from the flattened tensor
        # Map original lat/lon values to grid indices
        lat_map = {lat.item(): i for i, lat in enumerate(unique_lats)}
        lon_map = {lon.item(): i for i, lon in enumerate(unique_lons)}

        # Use the maps to get indices and reshape
        lat_indices = [lat_map[lat.item()] for lat in tensor[:, lat_col]]
        lon_indices = [lon_map[lon.item()] for lon in tensor[:, lon_col]]
        
        # Create the grid
        data_grid = torch.zeros((n1, n2), dtype=tensor.dtype)
        data_grid[lat_indices, lon_indices] = tensor[:, val_col]

        # Perform 2D FFT
        fft_results.append(torch.fft.fft2(data_grid))
    
    # Stack the 2D FFT results into a 3D tensor (n1, n2, p)
    J_vector_tensor = torch.stack(fft_results, dim=2)
    
    # Apply the normalization factor
    normalization_factor = 1.0 / (2 * cmath.pi)
    
    return J_vector_tensor * normalization_factor

def likelihood_vectorized(params, tensor_list):
    """
    Calculates the negative log-likelihood for the multivariate model
    using a vectorized approach for improved performance.
    
    Args:
        params (list): Model parameters.
        tensor_list (list): A list of tensors representing the multivariate data.
        
    Returns:
        torch.Tensor: The negative log-likelihood value.
    """
    p = len(tensor_list)
    if p == 0:
        return torch.tensor(0.0, dtype=torch.float32)
    
    # Determine dimensions from the first tensor
    unique_lats = torch.unique(tensor_list[0][:, 0])
    unique_lons = torch.unique(tensor_list[0][:, 1])
    n1 = len(unique_lats)
    n2 = len(unique_lons)
    n = n1 * n2

    # Compute the expected periodogram tensor from the model
    expected_periodogram = expected_periodogram_fft_multivariate(params, n1, n2, p)
    
    # Generate the J-vector tensor from the data
    j_vector = generate_Jvector(tensor_list)
    
    # Reshape tensors for batch processing
    I_omega_batch = expected_periodogram.reshape(-1, p, p)
    J_omega_batch = j_vector.reshape(-1, p, 1)

    # Add a small value to the real part of the diagonal for stability
    reg_term = torch.linalg.norm(I_omega_batch, dim=(1, 2), keepdim=True) * 1e-10 * torch.eye(p, dtype=I_omega_batch.dtype)
    I_omega_stable = torch.real(I_omega_batch) + reg_term
    
    # Compute the log-determinant of each matrix in the batch
    log_det = torch.log(torch.det(I_omega_stable)).real

    # Compute the inverse for each matrix in the batch
    inv_I_batch = torch.linalg.inv(I_omega_stable)
    
    # Compute the quadratic form: J^H * I^-1 * J
    term2_temp = torch.bmm(torch.conj(J_omega_batch.transpose(1, 2)), inv_I_batch)
    term2_batch = torch.bmm(term2_temp, J_omega_batch).reshape(-1)

    # The sum of log(det(I)) and the trace term over all frequencies
    nll_batch = log_det + term2_batch.real

    # Sum up all the log-likelihood terms
    nll = torch.sum(nll_batch)

    # Normalize the total negative log-likelihood
    return nll / n

# --- Example Usage with tensor_day1 ---

if __name__ == '__main__':
    # Assume tensor_day1 is your input tensor
    # To test the multivariate function, we'll split it into multiple time slices
    # Here, we'll create a list of 4 tensors, representing 4 time slices
    
    # Find unique time values
    unique_times = torch.unique(tensor_day1[:, 3])
    time_slices_list = []
    
    for t_val in unique_times:
        time_slices_list.append(tensor_day1[tensor_day1[:, 3] == t_val])

    print(f"Created a list of {len(time_slices_list)} tensors for multivariate test.")

    # Define some sample parameters
    params = torch.tensor([1.0, 1.0, 1.0, 0.01, 0.01, 0.01, 0.01], requires_grad=True)

# Calculate the negative log-likelihood
nll_value = likelihood_vectorized(params, time_slices_list)

print("\n--- Likelihood Calculation Complete ---")
print(f"Negative Log-Likelihood: {nll_value.item()}")




Created a list of 8 tensors for multivariate test.

--- Likelihood Calculation Complete ---
Negative Log-Likelihood: 6482987008.0


In [ ]:
from torch.optim.lr_scheduler import StepLR


# use adpating lr
def run_full(params, optimizer, scheduler,  covariance_function, epochs=10 ):
    
    """
    Run the training loop for the full likelihood model.

    Args:
        params (torch.Tensor): Model parameters.
        optimizer (torch.optim.Optimizer): Optimizer for updating parameters.
        scheduler (torch.optim.lr_scheduler): Learning rate scheduler.
        epochs (int): Number of epochs to train.

    Returns:
        list: Final parameters and loss.
        int: Number of epochs run.
    """

    prev_loss= float('inf')

    tol = 1e-3  # Convergence tolerance
    for epoch in range(epochs):  
        optimizer.zero_grad()  
        
        loss = likelihood_vectorized(params, time_slices_list)
        loss.backward()  # Backpropagate th e loss

        if epoch%100 == 0:
                # --- Check and print gradients and parameters at each iteration ---
            print(f'--- Epoch {epoch+1}/{epochs} ---')
            print(f' Loss: {loss.item()}')
            print(f' Parameters: {params.detach().numpy()}')
            print(f' Gradients: {params.grad}')
        
    
        # Check if loss is NaN or Inf, and break if it is
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Loss became NaN or Inf at epoch {epoch+1}. Stopping training.")
            print(f'Final Loss: {loss.item()}')
            print(f'Final Gradients: {params.grad}')
            break

        

        # Print gradients and parameters every 10th epoch
        #if epoch % 10 == 0:
        #    print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
        
        # if epoch % 500 == 0:
        #     print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
        
        optimizer.step()  
        scheduler.step()  
        # Check for convergence
        if abs(prev_loss - loss.item()) < tol:
            print(f"Converged at epoch {epoch}")
            print(f'Epoch {epoch+1}, : Loss: {loss.item()}, \n vecc Parameters: {params.detach().numpy()}')
            break

        prev_loss = loss.item()

    params = [torch.round(x*1000).detach().numpy()/1000 for x in params]
    loss = (torch.round(loss*1000)/1000).item()
    print(f'FINAL STATE: Epoch {epoch+1}, Loss: {loss}, \n vecc Parameters: {params}')
    return params + [loss], epoch

# Convert the list of floats to a PyTorch tensor
params = torch.tensor([20.0, 2.0, 3.0, 0.01, -0.001, 0.01, 2.01], requires_grad=True)
lr, betas, eps, step_size, gamma = 0.03, (0.9, 0.99), 1e-08, 40, 0.8
optimizer = torch.optim.Adam([params], lr=lr, betas=betas, eps=eps)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)  #

a,b = run_full( params, optimizer, scheduler, None, epochs=400 )

--- Epoch 1/400 ---
 Loss: 32286718.0
 Gradients: tensor([-1.0500e+01,  2.5297e+02,  3.9000e+01, -1.0750e+01,  3.2000e+01,
        -1.4188e+01, -1.6063e+07])
--- Epoch 101/400 ---
 Loss: 16558423.0
 Gradients: tensor([-1.6028e+03,  4.2513e+05, -5.2451e+03,  4.7245e+04,  9.2020e+03,
         1.1484e+04, -4.2256e+06])
--- Epoch 201/400 ---
 Loss: 15958496.0
 Gradients: tensor([-1.1995e+03,  3.0010e+05, -3.8144e+03,  3.8039e+04,  6.4340e+03,
         7.6995e+03, -3.9247e+06])
--- Epoch 301/400 ---
 Loss: 15490834.0
 Gradients: tensor([-1.1348e+03,  2.8400e+05, -3.6118e+03,  3.6053e+04,  6.0795e+03,
         7.2925e+03, -3.6980e+06])
FINAL STATE: Epoch 400, Loss: 15087420.0, 
 vecc Parameters: [21.688, 0.339, 2.632, 0.181, 0.296, 0.309, 4.295]


lr 0.01. step50 epoch 200

Loss: 22706554.000000
FINAL STATE: Epoch 200, Loss: 22706554.0, 
 vecc Parameters: [21.136, 0.79, 2.358, 0.122, -0.097, 0.0, 2.859]

 lr 0.01 step 100 epoch 600  smalll change after epoch 300

 --- Epoch 1/600 ---
 Loss: 32286718.0
 Gradients: tensor([-1.0500e+01,  2.5297e+02,  3.9000e+01, -1.0750e+01,  3.2000e+01,
        -1.4188e+01, -1.6063e+07])
--- Epoch 101/600 ---
 Loss: 22401716.0
 Gradients: tensor([-2.4288e+02,  3.1184e+04, -8.0125e+02, -2.1475e+02,  9.0000e+00,
        -9.9756e+00, -7.7329e+06])
--- Epoch 201/600 ---
 Loss: 17031722.0
 Gradients: tensor([ 2.6182e+05,  1.0712e+05, -1.9464e+06,  1.2475e+07,  1.0135e+08,
         8.4488e+06, -6.8022e+06])
--- Epoch 301/600 ---
 Loss: 16734517.0
 Gradients: tensor([ 3.2254e+05,  1.4013e+04, -4.5413e+05,  6.2497e+06,  2.3328e+07,
        -8.1825e+06, -6.9858e+06])
--- Epoch 401/600 ---
 Loss: 16704657.0
 Gradients: tensor([   249943.7500,     10871.4238,   -217640.0000,   7583911.0000,
           723587.0000, -10843837.0000,  -6504515.5000])
--- Epoch 501/600 ---
 Loss: 16663540.0
 Gradients: tensor([ 2.0924e+05,  9.9438e+03, -2.5152e+05,  7.5038e+06,  5.0710e+06,
        -1.1627e+07, -6.2263e+06])
FINAL STATE: Epoch 600, Loss: 16630042.0, 
 vecc Parameters: [21.645, -0.011, 3.064, -0.004, 0.007, 0.007, 3.407]

 lr 0.03 50 600

 --- Epoch 1/600 ---
 Loss: 32286718.0
 Gradients: tensor([-1.0500e+01,  2.5297e+02,  3.9000e+01, -1.0750e+01,  3.2000e+01,
        -1.4188e+01, -1.6063e+07])
--- Epoch 101/600 ---
 Loss: 17367230.0
 Gradients: tensor([    59566.8125, -20491166.0000,    -45886.5000, -17058900.0000,
         -2243651.7500,    468761.2500,  -5089638.0000])
--- Epoch 201/600 ---
 Loss: 16874694.0
 Gradients: tensor([   -8683.5625,  2789644.5000,   -95100.8750,  1474869.8750,
          567344.0000,   178033.7188, -4405246.0000])
--- Epoch 301/600 ---
 Loss: 16666024.0
 Gradients: tensor([   -7147.0938,  2270397.0000,   -86895.5000,   936746.6875,
          494532.1250,   173081.5625, -4305279.0000])
--- Epoch 401/600 ---
 Loss: 16588262.0
 Gradients: tensor([   -6682.1250,  2102722.5000,   -84959.6875,   796439.0000,
          478694.8750,   173921.0000, -4267694.5000])
--- Epoch 501/600 ---
 Loss: 16561779.0
 Gradients: tensor([   -6450.6562,  2013594.0000,   -84132.5000,   725261.8750,
          471525.3750,   174381.5938, -4255359.0000])
FINAL STATE: Epoch 600, Loss: 16553778.0, 
 vecc Parameters: [21.828, 0.184, 2.685, 0.073, 0.083, -0.104, 3.861]

In [50]:
from torch.optim.lr_scheduler import StepLR
# Convert the list of floats to a PyTorch tensor
params = torch.tensor([1.0, 1.0, 1.0, 0.01, 0.01, 0.01, 0.01], requires_grad=True)
lr, betas, eps, step_size, gamma = 0.1, (0.9, 0.99), 1e-08, 50, 0.5
optimizer = torch.optim.Adam([params], lr=lr, betas=betas, eps=eps)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)  #

def run_full(params, optimizer, scheduler, likelihood_function, epochs=10):
    """
    Run the training loop for the full likelihood model.

    Args:
        params (torch.Tensor): Model parameters.
        optimizer (torch.optim.Optimizer): Optimizer for updating parameters.
        scheduler (torch.optim.lr_scheduler): Learning rate scheduler.
        likelihood_function (callable): The likelihood function to be optimized.
        epochs (int): Number of epochs to train.

    Returns:
        tuple: Final parameters and loss, and the number of epochs run.
    """
    prev_loss = float('inf')
    tol = 1e-3  # Convergence tolerance
    
    # Store history for plotting
    loss_history = []
    param_history = []
    grad_history = []

    for epoch in range(epochs):  
        optimizer.zero_grad()  
        
        # Calculate loss using the provided likelihood function
        loss = likelihood_function(params)
        loss.backward()  # Backpropagate the loss
        
        # --- Check and print gradients and parameters at each iteration ---
        print(f'--- Epoch {epoch+1}/{epochs} ---')
        
        # Check if loss is NaN or Inf, and break if it is
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Loss became NaN or Inf at epoch {epoch+1}. Stopping training.")
            print(f'Final Loss: {loss.item()}')
            print(f'Final Gradients: {params.grad}')
            print(f'Final Parameters: {params.detach()}')
            break

        print(f'Loss: {loss.item():.6f}')
        print(f'Parameters: {params.detach().numpy()}')
        if params.grad is not None:
            print(f'Gradients: {params.grad.numpy()}')
            
            # Check for NaN gradients
            if torch.isnan(params.grad).any() or torch.isinf(params.grad).any():
                print(f"Gradients became NaN or Inf at epoch {epoch+1}. Stopping training.")
                break
        else:
            print('Gradients: Not computed yet (first epoch).')
        print("-" * 30)

        # Update parameters
        optimizer.step()  
        scheduler.step()  
        
        # Check for convergence
        if abs(prev_loss - loss.item()) < tol:
            print(f"\nConverged at epoch {epoch+1}")
            print(f'Final State: Loss: {loss.item():.6f}, \n Parameters: {params.detach().numpy()}')
            break

        prev_loss = loss.item()

    # Final print and return
    final_loss = loss.item()
    final_params = params.detach().numpy()
    
    print(f'\nFINAL STATE: Epoch {epoch+1}, Loss: {final_loss:.6f}, \n Parameters: {final_params}')
    
    return final_params, final_loss, epoch

a,b,c = run_full( params, optimizer, scheduler, likelihood_vectorized, epochs= 50
    )

TypeError: likelihood_vectorized() missing 1 required positional argument: 'tensor_list'